In [1]:
using JuMP, Gurobi

In [2]:
# 1 student, two activities (three including sleep) over one day. 
# Try to exclusively do activity 1 and 2, but required to do activity 3 for 10hrs

In [3]:
env = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


In [4]:
function one_student_one_day(μ; solver_outputs=0)
    
    model = Model(() -> Gurobi.Optimizer(env)) ;
    set_optimizer_attribute(model, "OutputFlag", solver_outputs)
    A = length(μ)
    
    # no need to index over students  (== 1)
    @variable(model, x[1:A, 1:48], Bin)
    
    # All half-hours must be occupied with an activity
    @constraint(model, sum(sum(x))==48)
    
    # Can only do one activity at a time
    @constraint(model, sum(x, dims=1).==1)
    
    # must do activity 3 for at least 10 hours / day
    @constraint(model, sum(x[3,:])>=10)
    
    # must sleep (activity 1) for at least 8 hours (not enforcing which hours atm)
    @constraint(model, sum(x[1,:])>=8)
    
    # objective is how we spend time weighted by value we assign to activity
    @objective(model, Max, sum(μ'*x))
    
    optimize!(model)
 
    return JuMP.value.(x)
end

one_student_one_day (generic function with 1 method)

In [5]:
μ = [1,2,-1]
day = one_student_one_day(μ)

3×48 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     1.0  1.0  0.0  1.0  1.0  1.0  0.0

In [9]:
function one_student(μ; solver_outputs=0)
    
    model = Model(() -> Gurobi.Optimizer(env)) ;
    set_optimizer_attribute(model, "OutputFlag", solver_outputs)
    A = length(μ)
    
    # no need to index over students  (== 1)
    @variable(model, x[1:A, 1:48, 1:7], Bin)
    
    # All half-hours must be occupied with an activity
    @constraint(model, [d=1:7], sum(sum(x[:,:,d]))==48)
    
    # Can only do one activity at a time
    @constraint(model, [d=1:7], sum(x[:,:,d], dims=1).==1)
    
    # must do activity 3 for at least 10 hours / day
    @constraint(model, [d=1:7], sum(x[3,:,d])>=10)
    
    # must sleep (activity 1) for at least 8 hours (not enforcing which hours atm)
    @constraint(model, [d=1:7], sum(x[1,:,d])>=8)
    
    # objective is how we spend time weighted by value we assign to activity
    @objective(model, Max, sum(μ'*reshape(sum(x, dims=3), (A, 48))))
    
    optimize!(model)
 
    return JuMP.value.(x)
end

one_student (generic function with 1 method)

In [10]:
μ = [1,2,-1]
week = one_student(μ)

3×48×7 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0     1.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0     1.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0     1.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0     1.0  0.0  0.0  1.0  1.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  0